In [1]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import numpy as np
import matplotlib.pyplot as plt

# 1. Generate toy dataset
def generate_data(n=1000):
    np.random.seed(42)
    x0 = np.random.randn(n // 2, 2) * 0.5 + np.array([1, 1])
    x1 = np.random.randn(n // 2, 2) * 0.5 + np.array([-1, -1])
    X = np.vstack([x0, x1])
    y = np.array([0] * (n // 2) + [1] * (n // 2)).reshape(-1, 1)
    return X, y

X_np, y_np = generate_data()
X = mx.array(X_np.astype(np.float32))
y = mx.array(y_np.astype(np.float32))

# 2. Define model
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid(),
        )

    def __call__(self, x):
        return self.net(x)

model = BinaryClassifier()
optimizer = optim.SGD(learning_rate=0.1)

# 3. Loss function: Binary Cross Entropy
def binary_cross_entropy(pred, target):
    eps = 1e-7
    pred = mx.clip(pred, eps, 1 - eps)
    return -(target * mx.log(pred) + (1 - target) * mx.log(1 - pred)).mean()

# 4. Training loop
for epoch in range(100):
    def loss_fn():
        y_pred = model(X)
        return binary_cross_entropy(y_pred, y)

    loss, grads = mx.value_and_grad(loss_fn)()
    optimizer.update(model, grads)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 5. Plot decision boundary
xx, yy = np.meshgrid(np.linspace(-3, 3, 200), np.linspace(-3, 3, 200))
grid = np.c_[xx.ravel(), yy.ravel()]
grid_mx = mx.array(grid.astype(np.float32))
probs = model(grid_mx).numpy().reshape(xx.shape)

plt.contourf(xx, yy, probs, levels=[0, 0.5, 1], alpha=0.3)
plt.scatter(X_np[:, 0], X_np[:, 1], c=y_np[:, 0], cmap="bwr", edgecolor="k")
plt.title("Decision Boundary")
plt.xlabel("x")
plt.ylabel("y")
plt.show()


ValueError: [value_and_grad] Can't compute the gradient of argument index 0 because the function is called with only 0 positional arguments.